In [12]:
%cd ~/work/deeplearning/

/home/asaeed9/work/deeplearning


## Headers

In [13]:
from theano.sandbox import cuda
#cuda.use('gpu1')
%matplotlib inline
from __future__ import print_function, division
#path="../data/2cat/sample"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink
from keras.preprocessing import image, sequence
import os, sys, cv2
from shutil import copyfile, move
import os
####
from keras.layers.convolutional import *
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
import datetime

In [14]:
data_dir="/home/asaeed9/work/data/2cat"
data_src = "sample"
path="/home/asaeed9/work/data/2cat/" + data_src + "/"
results_path = "/home/asaeed9/work/data/2cat/"+ data_src+"/results"
test_path = path + '/test/' #We use all the test data
os.chdir("/home/asaeed9/work/data/2cat/train")

## Data Preparation

In [38]:
#grey images - minimal image augmentation

#clean previous data
def adjust_prev_data_sample(dest):
    move_files(path + "train/cats", data_dir + "/" + dest, "*")
    move_files(path + "train/dogs", data_dir + "/" + dest, "*")
    move_files(path + "valid/cats", data_dir + "/" + dest, "*")
    move_files(path + "valid/dogs", data_dir + "/" + dest, "*")
    
#clean previous data
def adjust_prev_data(dest):
    move_files(data_dir + "/valid", data_dir + "/" + dest, "*")
    move_files(data_dir + "/used_train", data_dir + "/" + dest, "*")
    # %mv $data_dir/valid/* $data_dir/train/
    adjust_prev_data_sample(dest)

def copy_samples(train, validation, remove_existing):
    #print("Copying new samples for training...")
    #clean previous data
    if remove_existing:
        print("Adjusting previous data...")
        adjust_prev_data("train")

    #build validation set
#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(validation): os.rename(shuf[i], '../valid/' + shuf[i])

    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(train): os.rename(shuf[i], "../" + data_src + "/train/" + shuf[i])
    move_files(path + "train",path + "train/cats/","cat*.jpg")
    move_files(path + "train",path + "train/dogs/","dog*.jpg")

    # %mv ../sample/train/cat*.jpg ../sample/train/cats/
    # %mv ../sample/train/dog*.jpg ../sample/train/dogs/
#     os.chdir("../valid")
    #%cd ../valid

    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(validation): os.rename(shuf[i], "../" + data_src + "/valid/" + shuf[i])

    move_files(path + "valid",path + "valid/cats/","cat*.jpg")
    move_files(path + "valid",path + "valid/dogs/","dog*.jpg")
    #os.chdir(data_dir + "/train")
    
    #print(os.listdir(path+"train/dogs/"))
    dog_train_count = get_file_count(path + "train/dogs/") 
    dog_valid_count = get_file_count(path + "valid/dogs/")

    cat_train_count = get_file_count(path + "train/cats/") 
    cat_valid_count = get_file_count(path + "valid/cats/")
    
    return dog_train_count, dog_valid_count, cat_train_count, cat_valid_count

# def adjust_prev_data():
#     %mv $data_dir/valid/* $data_dir/train/
    
#     %rm $path/train/cats/*
#     %rm $path/train/dogs/*
#     %rm $path/valid/cats/*
#     %rm $path/valid/dogs/*

def move_files(src_path, dest_path, pattern):
    for file in glob(src_path + '/' + pattern):
        try:
                move(os.path.join(src_path+'/', os.path.basename(file)), os.path.join(dest_path+'/',os.path.basename(file)))
        except IOError, e:
                print ("Unable to move file. ".format(e))

def get_file_count(dir_path):
    return len([name for name in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path,name))])

#copy training images
# def copy_samples(train, validation):
    
#     #clean previous data
#     adjust_prev_data("train")
    
#     #build validation set
#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(validation): os.rename(shuf[i], '../valid/' + shuf[i]) 
    
#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(train): copyfile(shuf[i], '../sample/train/' + shuf[i]) 
#     %mv ../sample/train/cat*.jpg ../sample/train/cats/
#     %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
#     %cd ../valid

#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(validation): copyfile(shuf[i], '../sample/valid/' + shuf[i]) 
#     %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
#     %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
#     %cd $data_dir/train


def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

def predict(path, model, predict_type):
    gen_test = image.ImageDataGenerator()
    test_batches = gen_test.flow_from_directory(path+predict_type, class_mode=None, target_size=(256,256), shuffle=False, batch_size=1)
    test_data = np.concatenate([test_batches.next() for i in range(test_batches.nb_sample)])
    test_labels = onehot(test_batches.classes)
    score = model.evaluate(test_data, test_labels)
    
    probs = model.predict(test_data)
    
    #loss_score.append(score[0])
    #accuracy_score.append(score[1])
    
    #print("\nLoss:{}, Accuracy:{}".format(score[0], score[1]))
    #print("\nProbabilities:{}".format(probs))
    return probs, test_batches, score[0], score[1]

def pred_batch(imgs, classes):
    preds = model.predict(imgs)
    idxs = np.argmax(preds, axis=1)

    print('Shape: {}'.format(preds.shape))
    print('First 5 classes: {}'.format(classes[:5]))
    print('First 5 probabilities: {}\n'.format(preds[:5]))
    print('Predictions prob/class: ')
    
    for i in range(len(idxs)):
        idx = idxs[i]
        print ('  {:.4f}/{}'.format(preds[i, idx], classes[idx]))


def generate_size_graph(fig_no, training_size, accuracy, loss, start_size, end_size):
    plt.figure(fig_no,figsize=(7,5))
    plt.plot(training_size,accuracy)
    plt.plot(training_size,loss)
    plt.xlabel('Training Size')
    plt.ylabel('Accuracy/Loss')
    plt.title('Training Size vs Accuracy/Loss')
    plt.grid(True)
    plt.legend(['Accuracy','Loss'])
    plt.style.use(['classic'])
    plt.show()
    plt.savefig(path + '/batch_graphs/' +  str(start_size) + '_' + str(end_size) + '.jpg') 
        
def generate_graph(fig_no, epochs, train, val, label, train_title, val_title, train_size):
    plt.figure(fig_no,figsize=(7,5))
    plt.plot(epochs,train)
    plt.plot(epochs,val)
    plt.xlabel('num of Epochs')
    plt.ylabel(label)
    plt.title(train_title + ' vs ' + val_title + '( Samples:' + str(train_size) + ')')
    plt.grid(True)
    plt.legend(['train','val'])
    plt.style.use(['classic'])
    plt.show()
    plt.savefig(results_path + '/batch_graphs/' +  label + '_' + str(train_size) + '.jpg') 
    
def get_train_model(tr_batches, val_batches, epoch):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3, 256,256)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
#             Convolution2D(64,3,3, activation='relu'),
#             BatchNormalization(axis=1),
            #MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Flatten(),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(2, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=epoch - 2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

def train_model(model, tr_batches, val_batches, epoch):
    if not model:
        model = Sequential([
                BatchNormalization(axis=1, input_shape=(3, 256,256)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Dropout(0.2),
    #             Convolution2D(64,3,3, activation='relu'),
    #             BatchNormalization(axis=1),
                #MaxPooling2D((3,3)),
                Convolution2D(64,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Dropout(0.2),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Dropout(0.2),
                Flatten(),
                Dense(1024, activation='relu'),
                BatchNormalization(),
                Dropout(0.2),
                Dense(2, activation='softmax')
            ])
        model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=2, validation_data=val_batches,
                     nb_val_samples=val_batches.nb_sample)

    model.optimizer.lr = 0.1
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=1, validation_data=val_batches,
                     nb_val_samples=val_batches.nb_sample)

    model.optimizer.lr = 0.001
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=epoch - 3, validation_data=val_batches,
                     nb_val_samples=val_batches.nb_sample)

    return model


def fit(old_model, path, results_path, nepoch, batch_size, train_size, valid_size):
    gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05,
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

    #for train in training_range:
    model = None
    if old_model:
        dog_train_count, dog_valid_count, cat_train_count, cat_valid_count = copy_samples(train_size, valid_size, False)

        print("Train + Valid Dogs: {} \n Train + Valid Cats: {}".format(dog_train_count + 
                                                                        dog_valid_count,  
                                                                        cat_train_count + 
                                                                        cat_valid_count))  
    else:
        print("First iteration...")
        dog_train_count, dog_valid_count, cat_train_count, cat_valid_count = copy_samples(train_size, valid_size, True)

        print("Train + Valid Dogs: {} \n Train + Valid Cats: {}".format(dog_train_count + 
                                                                        dog_valid_count,  
                                                                        cat_train_count + 
                                                                        cat_valid_count))  
    
        
    tr_batches = gen_t.flow_from_directory(path + 'train', batch_size=batch_size)
    val_batches = gen_t.flow_from_directory(path + 'valid', class_mode='categorical', 
                                            shuffle=True, batch_size=batch_size * 2)

    if old_model:
        model = train_model(old_model, tr_batches, val_batches, nepoch)
    else:
        model = train_model(None, tr_batches, val_batches, nepoch)

    #model.summary()
    
    #model.save_weights(results_path+ '/' + 'ft_' + str(train_size) + '.e' + str(nepoch))
    last_file_timestamp = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())
    #print('File Time Stamp:{}'.format(last_file_timestamp))
    model.save_weights(results_path+'/ft_{}'.format(last_file_timestamp))
    #model.load_weights(results_path+'/ft_{}'.format(last_file_timestamp))

    return model, last_file_timestamp


def get_test_model():
    model = Sequential([
                BatchNormalization(axis=1, input_shape=(3,256,256)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
    #             Convolution2D(64,3,3, activation='relu'),
    #             BatchNormalization(axis=1),
                #MaxPooling2D((3,3)),
                Convolution2D(64,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Flatten(),
                Dense(1024, activation='relu'),
                BatchNormalization(),
                Dense(2, activation='softmax')
            ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])  
    
    return model

### Training

In [39]:
os.chdir(data_dir + "/train") 
training_set_size = []
loss_array = []
accuracy_array = []
loss=0.0
accuracy=0.0
nepoch = 5
batch_size = 64
training_range = range(100, 30000, 2000)
tr_model = None
#%rm $path/results/*

for train_size in training_range:
    model = None
    #train = 15000
    valid_size = int(math.floor(.2 * train_size))
    print('Train size: {}'.format(train_size))
    print('Valid size: {}'.format(valid_size))
    
#    copy_samples(train, valid)

#     tr_batches = gen_t.flow_from_directory(path + 'train', batch_size=batch_size)
#     val_batches = gen_t.flow_from_directory(path + 'valid', class_mode='categorical', shuffle=True, batch_size=batch_size * 2)

#     model = get_train_model(tr_batches, val_batches, nepoch)
    
#     model.save_weights(results_path+ '/' + 'ft_' + str(train) + '.e' + str(nepoch))

    tr_model,file_timestamp = fit(tr_model, path, results_path, nepoch, 
                                  batch_size, train_size, valid_size)
    
    print("\nVerification on Test set.")
    model_test = None
    model_test = get_test_model()
    #model.load_weights(results_path+'/ft_' + str(train_size) + '.e' + str(nepoch))
    #print('{0}/ft_{1}'.format(last_file_timestamp))
    #print('Last File Timestamp- before loading:{}'.format(file_timestamp))
    model_test.load_weights(results_path+'/ft_{}'.format(file_timestamp))

    probs, test_batches,loss, accuracy = predict(path, model_test, "test")
    training_set_size.append(train_size)
    loss_array.append(loss)
    accuracy_array.append(accuracy)

    print('\nTraining Set Size:{}'.format(training_set_size))
    print('Accuracy:{}'.format(accuracy_array))
    print('Loss:{}'.format(loss_array))

Train size: 100
Valid size: 20
First iteration...
Adjusting previous data...
[]
Path: /home/asaeed9/work/data/2cat/sample/train/dogs/, File Count: 53
Train + Valid Dogs: 63 
 Train + Valid Cats: 57
Found 100 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


KeyboardInterrupt: 

## Prediction

In [ ]:
#training_range = range(3000, 18001, 3000)

In [ ]:
model = None
model = get_test_model()
loss_score = []
accuracy_score = []
training_size = training_range

os.chdir(results_path)
files = filter(os.path.isfile, os.listdir(results_path))
files.sort(key=lambda x: os.path.getmtime(x))

for f in files:
    
    print("\nFile being processed: {}".format(f))
    
    train_size = int(f[f.find('_')+1 : f.find('.')])
    epoch = int(f[f.find('e')+1 : ])

    model.load_weights(results_path+'/ft_' + str(train_size) + '.e' + str(epoch))
    
    gen_test = image.ImageDataGenerator()
    test_batches = gen_test.flow_from_directory(path+'test', class_mode=None, target_size=(256,256), shuffle=False, batch_size=1)
    test_data = np.concatenate([test_batches.next() for i in range(test_batches.nb_sample)])
    test_labels = onehot(test_batches.classes)
    score = model.evaluate(test_data, test_labels)
    
    loss_score.append(score[0])
    accuracy_score.append(score[1])
    
    print("\nLoss:{}, Accuracy:{}".format(score[0], score[1]))

#print(loss_score, accuracy_score)
#fig_no, training_size, accuracy, loss, start_size, end_size
generate_size_graph(1, training_size, accuracy_score, loss_score, min(training_size), max(training_size))

In [ ]:
training_size = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1500, 2000, 2500, 3000, 6000, 9000, 12000, 15000, 18000]
accuracy_score = [0.53033,0.53033,0.648333, 0.569333, 0.638666, 0.59800, 0.6416667, 0.7143333, 0.6746667, 0.6569999,0.6846667, 0.6756667, 0.7140000, 0.7703333,  0.82533, 0.871, 0.87966, 0.920333, 0.924]
loss_score = [1.77806968212, 2.08722846731, 1.21485737689, 1.99998559408, 1.04420828152, 1.76575960827, 1.2628258756, 0.975768786112, 1.28816328855, 1.22704125532, 1.395495, 1.239842, 0.921525, 0.7399451, 0.580999, 0.342369, 0.33888, 0.233011, 0.2023495]

print(len(training_size), len(accuracy_score), len(loss_score))

In [ ]:
generate_size_graph(1, training_size, accuracy_score, loss_score, min(training_size), max(training_size))

In [ ]:
def get_numpy_images(data_path):
    data_dir_list = os.listdir(data_path)
    img_data_list=[]
    labels=[]

    for dataset in data_dir_list:
        img_list=os.listdir(data_path+'/'+ dataset)
        print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
        for img in img_list:

            if 'cat' in img:
                labels.append(0)
            else:
                labels.append(1)

            input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
            #input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
            print(type(input_img), input_img.shape)
            input_img_resize=cv2.resize(input_img,(256,256))
            print(type(input_img_resize), input_img_resize.shape)
            exit(1)
            img_data_list.append(input_img_resize)

    img_data = np.array(img_data_list)
    img_data = img_data.astype('float32')
    return img_data, labels

In [ ]:
test_img_data, test_labels = get_numpy_images(test_path)
#test_img_data = np.expand_dims(test_img_data, axis=1)
test_nsamples = test_img_data.shape[0]
test_input_shape=test_img_data[0].shape

test_Y = np_utils.to_categorical(test_labels, 2)

In [ ]:
score = model.evaluate(test_img_data, test_Y, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model.predict_classes()

In [ ]:
filenames = test_batches.filenames
np.set_printoptions(suppress=True, precision=4)
print(filenames[:10])
print(preds[:10])
#our_labels = np.round(preds[:, 0])
#our_labels[:10]
#our_preds = preds[:, 1]

In [ ]:
#print(preds[:10])
filenames = test_batches.filenames
np.set_printoptions(suppress=True, precision=4)
filenames[:10]

In [ ]:
expected_labels = get_test_labels(test_batches.filenames)